In [13]:
import lmdb
import os
import copy
import torch
import random
import skimage
import numpy as np
import torch.nn as nn
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

print("complete")

complete


In [14]:
train_vectors = np.load("train_vectors_ave1.npy")
test_vectors = np.load("test_vectors_ave1.npy")

In [15]:
def pro_select(vectors, probs, num):
    points = np.zeros((num, 2))
    core_point = np.zeros((1, 2))
    for i in range(num):
        x = random.uniform(0, 1)
        cumprob = 0.0
        k = -1
        while x > cumprob:
            k += 1
            cumprob += probs[k]
        points[i, 0], points[i, 1] = vectors[k, 0], vectors[k, 1]
        
        adj_list = []
        core_point[0, 0], core_point[0, 1] = vectors[k, 0], vectors[k, 1]
        dis = np.sum(pow(vectors - core_point, 2), axis=1)
        for j in range(len(dis)):
            if dis[j] <= 8:
                adj_list.append(j)
        for j in range(len(adj_list)):
            probs[adj_list[j]] /= 100.0
            
        probs[k] = 0
        probs = probs/np.sum(probs)
        
    return points

class Bird_Test_Process(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train_file, root_dir, train_vectors, num, transform=None):
        f = open(train_file, 'r')
        self.train_list = f.readlines()
        f.close()
        self.root_dir = root_dir
        self.transform = transform
        self.train_vectors = train_vectors
        self.part_num = num

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, idx):
        line = self.train_list[idx]
        img_dir_label = line.strip('\n').split(' ')
        img_dir = os.path.join(self.root_dir, img_dir_label[0])
        image = io.imread(img_dir)
        if len(image.shape) == 2:
            image = image[:,:,np.newaxis]
            image = np.tile(image, [1, 1, 3])
            
        image = skimage.util.img_as_ubyte(transform.resize(image, (448, 448))) 
        label = int(img_dir_label[1])
        sample = {}
        
        vectors = self.train_vectors[idx]

        points = pro_select(vectors[:, :2].astype(int), copy.deepcopy(vectors[:, 3]), self.part_num)

        points1 = pro_select(vectors[:, :2].astype(int), copy.deepcopy(vectors[:, 3]), self.part_num)

        points2 = pro_select(vectors[:, :2].astype(int), copy.deepcopy(vectors[:, 3]), self.part_num)

        for i in range(self.part_num):
            x, y = points[i][0] * 16 + 8, points[i][1] * 16 + 8
            l = random.randint(46, 98)
#             l = 46
            m1, m2, n1, n2 = int(np.maximum(0, x - l)), int(np.minimum(448, x + l)), \
                            int(np.maximum(0, y - l)), int(np.minimum(448, y + l))
            sample['image%d' % i] = image[m1:m2, n1:n2]
            x1, y1 = points1[i][0] * 16 + 8, points1[i][1] * 16 + 8
            l1 = random.randint(46, 98)
            m1, m2, n1, n2 = int(np.maximum(0, x1 - l1)), int(np.minimum(448, x1 + l1)), \
                            int(np.maximum(0, y1 - l1)), int(np.minimum(448, y1 + l1))
            sample['s_image%d' % i] = image[m1:m2, n1:n2]
            x2, y2 = points2[i][0] * 16 + 8, points2[i][1] * 16 + 8
            l2 = random.randint(46, 98)
            m1, m2, n1, n2 = int(np.maximum(0, x2 - l2)), int(np.minimum(448, x2 + l2)), \
                            int(np.maximum(0, y2 - l2)), int(np.minimum(448, y2 + l2))
            sample['t_image%d' % i] = image[m1:m2, n1:n2]
            
        sample['label'] = label

        if self.transform:
            for i in range(self.part_num):
                sample['image%d' % i] = self.transform(sample['image%d' % i])
                sample['s_image%d' % i] = self.transform(sample['s_image%d' % i])
                sample['t_image%d' % i] = self.transform(sample['t_image%d' % i])

        return sample
    
class Bird_Dataset_Process(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train_file, root_dir, train_vectors, num, transform=None):
        f = open(train_file, 'r')
        self.train_list = f.readlines()
        f.close()
        self.root_dir = root_dir
        self.transform = transform
        self.train_vectors = train_vectors
        self.part_num = num

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, idx):
        line = self.train_list[idx]
        img_dir_label = line.strip('\n').split(' ')
        img_dir = os.path.join(self.root_dir, img_dir_label[0])
        image = io.imread(img_dir)
        if len(image.shape) == 2:
            image = image[:,:,np.newaxis]
            image = np.tile(image, [1, 1, 3])
            
        image = skimage.util.img_as_ubyte(transform.resize(image, (448, 448))) 
        label = int(img_dir_label[1])
        sample = {}
        
        vectors = self.train_vectors[idx]
        points = pro_select(vectors[:, :2].astype(int), vectors[:, 3], self.part_num)
        for i in range(self.part_num):
            x, y = points[i][0] * 16 + 8, points[i][1] * 16 + 8
            l = random.randint(46, 98)
#             l = 46
            m1, m2, n1, n2 = int(np.maximum(0, x - l)), int(np.minimum(448, x + l)), \
                            int(np.maximum(0, y - l)), int(np.minimum(448, y + l))
            sample['image%d' % i] = image[m1:m2, n1:n2]
        
        sample['label'] = label

        if self.transform:
            for i in range(self.part_num):
                sample['image%d' % i] = self.transform(sample['image%d' % i])

        return sample

In [16]:
data_transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize(224),
                transforms.RandomCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.486,0.499,0.432], [0.229,0.225,0.263])
            ])

root_dir = '/home/lvlab/Pytorch/images'
train_file = '/home/lvlab/Pytorch/train.txt'
test_file = '/home/lvlab/Pytorch/test.txt'

img_data = {'train': Bird_Dataset_Process(train_file, root_dir, train_vectors, 4, data_transform),
                  'test': Bird_Test_Process(test_file, root_dir, test_vectors, 4, data_transform)}
loader_img = {x: torch.utils.data.DataLoader(img_data[x], batch_size=20,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}

dataset_sizes = {x: len(img_data[x]) for x in ['train', 'test']}
print(img_data['test'][0])

{'image2': tensor([[[-1.0092, -1.0263, -0.9920,  ..., -2.0880, -2.0709, -2.0538],
         [-0.9407, -0.9920, -1.0092,  ..., -2.0880, -2.0709, -2.0538],
         [-0.9064, -0.9578, -0.9749,  ..., -2.0880, -2.0709, -2.0538],
         ...,
         [ 0.3779,  0.3779,  0.3951,  ..., -0.0502,  0.1382,  0.2581],
         [ 0.5321,  0.4293,  0.3437,  ..., -0.3584, -0.1187,  0.0526],
         [ 0.6348,  0.4636,  0.3094,  ..., -0.6153, -0.3242, -0.1187]],

        [[-1.3986, -1.4160, -1.3812,  ..., -0.6666, -0.6666, -0.6492],
         [-1.3289, -1.3812, -1.3986,  ..., -0.6492, -0.6492, -0.6317],
         [-1.2940, -1.3463, -1.3812,  ..., -0.6492, -0.6492, -0.6317],
         ...,
         [-0.1437, -0.1437, -0.1437,  ..., -0.6143, -0.5097, -0.4226],
         [ 0.0132, -0.0914, -0.1786,  ..., -0.8060, -0.6317, -0.5271],
         [ 0.1177, -0.0391, -0.1960,  ..., -0.9803, -0.7537, -0.6143]],

        [[-1.0312, -1.0611, -1.0611,  ...,  0.7133,  0.7133,  0.7282],
         [-0.9567, -1.0163, -1.061

In [17]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
                
        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            start_time = time.time()        
            if epoch % 5 == 0 and phase == 'train':
                for param_group in optimizer.param_groups:
                    lr = param_group['lr']
                    print("***********************")
                    print("learning rate = %f" % lr)
            
            # Iterate over data.
            for i_batch, sample_batched in enumerate(loader_img[phase]):
                img0 = sample_batched['image0']
                img1 = sample_batched['image1']
                img2 = sample_batched['image2']
                img3 = sample_batched['image3']
                labels = sample_batched['label']
                if phase == 'test':
                    s_img0 = sample_batched['s_image0']
                    s_img1 = sample_batched['s_image1']
                    s_img2 = sample_batched['s_image2']
                    s_img3 = sample_batched['s_image3']
                    t_img0 = sample_batched['t_image0']
                    t_img1 = sample_batched['t_image1']
                    t_img2 = sample_batched['t_image2']
                    t_img3 = sample_batched['t_image3']
                    s_img0 = s_img0.to(device)
                    s_img1 = s_img1.to(device)
                    s_img2 = s_img2.to(device)
                    s_img3 = s_img3.to(device)
                    t_img0 = t_img0.to(device)
                    t_img1 = t_img1.to(device)
                    t_img2 = t_img2.to(device)
                    t_img3 = t_img3.to(device)
                    
                img0 = img0.to(device)
                img1 = img1.to(device)
                img2 = img2.to(device)
                img3 = img3.to(device)
                labels = labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    if phase == 'train':
                        output0 = model(img0)
                        output1 = model(img1)
                        output2 = model(img2)
                        output3 = model(img3)
                    else:
                        output0 = (model(img0) + model(s_img0) + model(t_img0))/3.0
                        output1 = (model(img1) + model(s_img1) + model(t_img1))/3.0
                        output2 = (model(img2) + model(s_img2) + model(t_img2))/3.0
                        output3 = (model(img3) + model(s_img3) + model(t_img3))/3.0
                    outputs = output0 + output1 + output2 + output3
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * img3.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                if i_batch % 50 == 0 and phase == 'train':
                    print("Iteration %d, loss = %f" % (i_batch, loss))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'test':
                scheduler.step(epoch_loss)
                
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            cost_time = (time.time() - start_time)/60.0
            print('{} 1 epoch time: {:.2f}min'.format(
                phase, cost_time))
            
            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [18]:
import time
import copy
import warnings
import torch.optim as optim
from torch.optim import lr_scheduler

warnings.filterwarnings('ignore')

device = torch.device("cuda:1")

vgg_16 = models.vgg16(pretrained=True)
vgg_16.classifier = nn.Sequential(*list(vgg_16.classifier.children())[:1])
vgg_16.features[30] = nn.AvgPool2d(kernel_size=14, stride=1, padding=0)
vgg_16.features.add_module('31', nn.Dropout(0.8)) 
vgg_16.classifier[0] = nn.Linear(in_features=512, out_features=200, bias=True)
# vgg_16 = nn.DataParallel(vgg_16, device_ids=[2, 3])
vgg_16 = vgg_16.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(vgg_16.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, mode='min', factor=0.5, patience=5)

model = train_model(vgg_16, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=60)

Epoch 0/59
----------
***********************
learning rate = 0.001000
Iteration 0, loss = 5.609405
Iteration 50, loss = 5.340823
Iteration 100, loss = 5.325879
Iteration 150, loss = 5.218789
Iteration 200, loss = 5.298266
Iteration 250, loss = 5.295120
train Loss: 5.2862 Acc: 0.0082
train 1 epoch time: 4.00min
test Loss: 5.0840 Acc: 0.0343
test 1 epoch time: 3.97min

Epoch 1/59
----------
Iteration 0, loss = 5.030942
Iteration 50, loss = 5.073341
Iteration 100, loss = 4.260230
Iteration 150, loss = 4.634339
Iteration 200, loss = 4.496074
Iteration 250, loss = 4.314782
train Loss: 4.6899 Acc: 0.0581
train 1 epoch time: 4.01min
test Loss: 3.7128 Acc: 0.2114
test 1 epoch time: 3.98min

Epoch 2/59
----------
Iteration 0, loss = 3.956013
Iteration 50, loss = 3.668451
Iteration 100, loss = 3.241007
Iteration 150, loss = 4.105084
Iteration 200, loss = 3.205766
Iteration 250, loss = 3.676556
train Loss: 3.5129 Acc: 0.2005
train 1 epoch time: 4.01min
test Loss: 2.4915 Acc: 0.4260
test 1 epoch 

train Loss: 0.4141 Acc: 0.8735
train 1 epoch time: 3.89min
test Loss: 0.9345 Acc: 0.7803
test 1 epoch time: 3.57min

Epoch 25/59
----------
***********************
learning rate = 0.001000
Iteration 0, loss = 0.355303
Iteration 50, loss = 0.261305
Iteration 100, loss = 0.201091
Iteration 150, loss = 0.706850
Iteration 200, loss = 0.226842
Iteration 250, loss = 0.196595
train Loss: 0.3829 Acc: 0.8827
train 1 epoch time: 3.92min
test Loss: 0.9997 Acc: 0.7742
test 1 epoch time: 3.62min

Epoch 26/59
----------
Iteration 0, loss = 0.072908
Iteration 50, loss = 0.184351
Iteration 100, loss = 0.548469
Iteration 150, loss = 0.729292
Iteration 200, loss = 0.283385
Iteration 250, loss = 0.461755
train Loss: 0.3759 Acc: 0.8866
train 1 epoch time: 3.92min
test Loss: 1.0982 Acc: 0.7658
test 1 epoch time: 3.62min

Epoch 27/59
----------
Iteration 0, loss = 0.043780
Iteration 50, loss = 0.349546
Iteration 100, loss = 0.318291
Iteration 150, loss = 0.725532
Iteration 200, loss = 0.171031
Iteration 250

Iteration 100, loss = 0.039018
Iteration 150, loss = 0.011755
Iteration 200, loss = 0.168095
Iteration 250, loss = 0.047929
train Loss: 0.0618 Acc: 0.9795
train 1 epoch time: 3.84min
test Loss: 1.0724 Acc: 0.8198
test 1 epoch time: 3.45min

Epoch 50/59
----------
***********************
learning rate = 0.000063
Iteration 0, loss = 0.196656
Iteration 50, loss = 0.218981
Iteration 100, loss = 0.041288
Iteration 150, loss = 0.011763
Iteration 200, loss = 0.036086
Iteration 250, loss = 0.005183
train Loss: 0.0684 Acc: 0.9775
train 1 epoch time: 3.87min
test Loss: 1.0740 Acc: 0.8233
test 1 epoch time: 3.45min

Epoch 51/59
----------
Iteration 0, loss = 0.049425
Iteration 50, loss = 0.041713
Iteration 100, loss = 0.279963
Iteration 150, loss = 0.058971
Iteration 200, loss = 0.050110
Iteration 250, loss = 0.022491
train Loss: 0.0601 Acc: 0.9810
train 1 epoch time: 3.86min
test Loss: 1.0898 Acc: 0.8210
test 1 epoch time: 3.47min

Epoch 52/59
----------
Iteration 0, loss = 0.017631
Iteration 50

In [20]:
torch.save(model, "random_part_bird")